In [ ]:
# key = 'YOUR_KEY_HERE'  
# endpoint = 'YOUR_ENDPOINT_HERE'  

In [3]:
import asyncio  
import nest_asyncio  
  
nest_asyncio.apply()  
  
async def sample_recognize_entities_async() -> None:  
    print(  
        "이 샘플에서 우리는 케이터링 업체이며 리뷰를 정렬하려고 합니다."  
        "케이터링을 위해 우리를 고용 한 조직을 기반으로하는 우리 조직의 경우"  
    )  
    import typing  
    from azure.core.credentials import AzureKeyCredential  
    from azure.ai.textanalytics.aio import TextAnalyticsClient  
  
    # key = 'YOUR_KEY_HERE'  
    # endpoint = 'YOUR_ENDPOINT_HERE'  
  
    text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))  
  
    reviews = [  
        """저는 푸 컴퍼니에서 일하고 있고, 매년 열리는 창립식에 콘토소를 고용했습니다. 음식이  
        정말 대단했고, 우리 모두는 품질과 서비스 수준에 대해 충분한 좋은 말을 할 수 없습니다.""",  
        """우리는 푸 컴퍼니에서 과거 회사와의 모든 성공 후에 콘토소를 다시 고용했습니다.  
        음식은 여전히 훌륭했지만 지난번보다 품질이 떨어졌다는 느낌이 듭니다  
        우리를 위해 음식을 제공합니다. 같은 문제에 부딪히는 사람이 또 있나요?""",  
        """최고의 슬라이더인 콘토소로부터 받은 서비스에 대해서는 바르사가 하늘을 찌르고 있다!!!"""  
    ]  
  
    async with text_analytics_client:  
        result = await text_analytics_client.recognize_entities(reviews)  
  
    result = [review for review in result if not review.is_error]  
  
    organization_to_reviews: typing.Dict[str, typing.List[str]] = {}  
    for idx, review in enumerate(result):  
        for entity in review.entities:  
            print(f"Entity '{entity.text}' has category '{entity.category}'")  
            if entity.category == 'Organization':  
                organization_to_reviews.setdefault(entity.text, [])  
                organization_to_reviews[entity.text].append(reviews[idx])  
  
    for organization, reviews in organization_to_reviews.items():  
        print(  
            "\n\nOrganization '{}' has left us the following review(s): {}".format(  
                organization, "\n\n".join(reviews)  
            )  
        )  
  
async def main():  
    await sample_recognize_entities_async()  
  
# Jupyter 노트북에서는 아래와 같이 직접 호출  
await main()  

이 샘플에서 우리는 케이터링 업체이며 리뷰를 정렬하려고 합니다.케이터링을 위해 우리를 고용 한 조직을 기반으로하는 우리 조직의 경우
Entity '푸 컴퍼니' has category 'Organization'
Entity '매년' has category 'DateTime'
Entity '창립식' has category 'Event'
Entity '음식' has category 'Product'
Entity '서비스' has category 'Skill'
Entity '푸 컴퍼니' has category 'Organization'
Entity '콘토소' has category 'Person'
Entity '음식' has category 'Product'
Entity '음식' has category 'Product'
Entity '사람이' has category 'PersonType'
Entity '슬라이더' has category 'PersonType'
Entity '콘토소' has category 'Person'
Entity '바르사가' has category 'Person'


Organization '푸 컴퍼니' has left us the following review(s): 저는 푸 컴퍼니에서 일하고 있고, 매년 열리는 창립식에 콘토소를 고용했습니다. 음식이  
        정말 대단했고, 우리 모두는 품질과 서비스 수준에 대해 충분한 좋은 말을 할 수 없습니다.

우리는 푸 컴퍼니에서 과거 회사와의 모든 성공 후에 콘토소를 다시 고용했습니다.  
        음식은 여전히 훌륭했지만 지난번보다 품질이 떨어졌다는 느낌이 듭니다  
        우리를 위해 음식을 제공합니다. 같은 문제에 부딪히는 사람이 또 있나요?


In [4]:
import asyncio
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics.aio import TextAnalyticsClient

# 새로 생성한 함수: reviews를 나눠서 여러 번 호출
async def recognize_entities_in_batches(client, reviews_batches):
    tasks = []
    for batch in reviews_batches:
        tasks.append(client.recognize_entities(documents=batch))
    return await asyncio.gather(*tasks)

# 원래 함수 수정: 리뷰를 나눠서 recognize_entities_in_batches 함수 호출
async def recognize_entities_async() -> None:
    print("이 샘플에서 우리는 케이터링 업체이며 리뷰를 정렬하려고 합니다.")

    text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))
    reviews = [
        """저는 푸 컴퍼니에서 일하고 있고, 매년 열리는 창립식에 콘토소를 고용했습니다. 음식이 정말 대단했고, 우리 모두는 품질과 서비스 수준에 대해 충분한 좋은 말을 할 수 없습니다.""",
        """우리는 푸 컴퍼니에서 과거 회사와의 모든 성공 후에 콘토소를 다시 고용했습니다. 음식은 여전히 훌륭했지만 지난번보다 품질이 떨어졌다는 느낌이 듭니다. 우리를 위해 음식을 제공합니다. 같은 문제에 부딪히는 사람이 또 있나요?""",
        """최고의 슬라이더인 콘토소로부터 받은 서비스에 대해서는 바르사가 하늘을 찌르고 있다!!!""",
        """우리는 오메가 사에서 콘토소를 여러 번 고용했습니다. 항상 훌륭한 음식과 서비스를 제공합니다. 직원들은 매우 전문적이며, 우리 손님들은 항상 만족합니다.""",
        """콘토소의 음식은 매번 최고입니다. 작년 행사에 이어 올해도 그들의 서비스를 이용했는데, 이번에도 실망시키지 않았습니다. 푸 컴퍼니가 주최한 이벤트에서 매우 성공적이었습니다.""",
        """바르사 회사에서 열린 최근 행사에서 콘토소를 고용했습니다. 음식과 서비스는 기대 이상이었고, 손님들로부터 많은 칭찬을 받았습니다. 다시 고용할 것입니다.""",
        """저는 푸 컴퍼니에서 일하고 있고, 지난 5년간 콘토소를 매년 고용했습니다. 그들의 음식과 서비스는 항상 일관되게 훌륭합니다.""",
        """콘토소의 음식은 훌륭하지만, 최근 이벤트에서는 약간의 실수가 있었습니다. 오메가 사의 직원들 사이에서 약간의 불만이 있었습니다. 그러나 전반적으로 여전히 좋은 서비스입니다.""",
        """우리는 바르사 회사에서 콘토소를 고용하여 최근 행사를 진행했습니다. 음식은 환상적이었고, 모든 손님들이 만족했습니다. 다시 한번 고용할 계획입니다.""",
        """푸 컴퍼니에서 열린 큰 행사에서 콘토소를 고용했습니다. 음식과 서비스 모두 완벽했습니다. 손님들은 매우 만족했습니다.""",
        """오메가 사에서 열린 이벤트에서 콘토소를 고용했습니다. 음식은 좋았지만, 서비스가 조금 느렸습니다. 개선되었으면 좋겠습니다.""",
        """콘토소의 음식은 항상 최고입니다. 바르사 회사에서 열린 모든 행사에서 그들을 고용했습니다. 손님들로부터 많은 칭찬을 받았습니다.""",
        """푸 컴퍼니에서 열린 이벤트에서 콘토소를 다시 고용했습니다. 음식은 훌륭했지만, 이번에는 서비스가 조금 느렸습니다. 그러나 여전히 좋은 경험이었습니다.""",
        """오메가 사에서 열린 작은 행사에서 콘토소를 고용했습니다. 음식과 서비스 모두 훌륭했습니다. 손님들은 매우 만족했습니다.""",
        """바르사 회사에서 열린 큰 행사에서 콘토소를 고용했습니다. 음식과 서비스 모두 완벽했습니다. 손님들은 매우 만족했습니다.""",
        """푸 컴퍼니에서 열린 연말 행사에서 콘토소를 고용했습니다. 음식은 훌륭했지만, 서비스가 약간 느렸습니다. 그러나 전반적으로 좋은 경험이었습니다.""",
        """오메가 사에서 열린 행사에서 콘토소를 고용했습니다. 음식과 서비스 모두 좋았습니다. 손님들은 매우 만족했습니다.""",
        """바르사 회사에서 열린 이벤트에서 콘토소를 다시 고용했습니다. 음식은 훌륭했지만, 서비스가 조금 느렸습니다. 그러나 여전히 좋은 경험이었습니다.""",
        """푸 컴퍼니에서 열린 행사에서 콘토소를 고용했습니다. 음식과 서비스 모두 좋았습니다. 손님들은 매우 만족했습니다.""",
        """오메가 사에서 열린 작은 행사에서 콘토소를 고용했습니다. 음식은 훌륭했지만, 서비스가 약간 느렸습니다. 그러나 전반적으로 좋은 경험이었습니다."""
    ]

    # 리뷰를 작은 배치로 나누기 (여기서는 5개씩 나눔)
    batch_size = 5
    reviews_batches = [reviews[i:i + batch_size] for i in range(0, len(reviews), batch_size)]

    async with text_analytics_client:
        results = await recognize_entities_in_batches(text_analytics_client, reviews_batches)

    organization_to_reviews = {}

    for result in results:
        for idx, review in enumerate(result):
            if not review.is_error:
                for entity in review.entities:
                    print(f"Entity '{entity.text}' has category '{entity.category}'")
                    if entity.category == 'Organization':
                        organization_to_reviews.setdefault(entity.text, [])
                        organization_to_reviews[entity.text].append(reviews[idx])

    for organization, reviews in organization_to_reviews.items():
        print(f"\n\nOrganization '{organization}' has left us the following review(s): {reviews}")

async def main():
    await recognize_entities_async()

if __name__ == '__main__':
    asyncio.run(main())


이 샘플에서 우리는 케이터링 업체이며 리뷰를 정렬하려고 합니다.
Entity '푸 컴퍼니' has category 'Organization'
Entity '매년' has category 'DateTime'
Entity '창립식' has category 'Event'
Entity '음식' has category 'Product'
Entity '푸 컴퍼니' has category 'Organization'
Entity '콘토소' has category 'Person'
Entity '음식' has category 'Product'
Entity '음식' has category 'Product'
Entity '사람이' has category 'PersonType'
Entity '슬라이더' has category 'PersonType'
Entity '콘토소' has category 'Person'
Entity '바르사가' has category 'Person'
Entity '오메가 사' has category 'Organization'
Entity '음식' has category 'Product'
Entity '서비스를' has category 'Skill'
Entity '직원들' has category 'PersonType'
Entity '손님' has category 'PersonType'
Entity '콘토소' has category 'Location'
Entity '작년' has category 'DateTime'
Entity '올해' has category 'DateTime'
Entity '푸 컴퍼니' has category 'Organization'
Entity '바르' has category 'Location'
Entity '행사' has category 'Event'
Entity '콘' has category 'Person'
Entity '소' has category 'Person'
Entity '손님' has category 'PersonType'
Ent